## Run this notebook

You can launch this notebook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=example-notebooks/nceo-biomass-statistics.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. 

## About the Data

[NCEO Aboveground Woody Biomass 2017](https://ceos.org/gst/africa-biomass.html) is map for the year 2017 at 100 m spatial resolution which was developed using a combination of LiDAR, Synthetic Aperture Radar (SAR) and optical based data. Aboveground woody biomass (AGB) plays an key role in the study of the Earth's carbon cycle and response to climate change. It is expressed as dry matter in Mg ha-1 and estimation based on Earth Observation measurements is an effective method for regional scale studies. 

**Important note**: Users of this dataset should keep in mind that the map is a continental-scale dataset, generated using a combination of different remote sensing data types, with a single method for the whole study area. Therefore, users should understand that accuracy may vary for different regions and vegetation types.

## The Case Study - 

TBD

## Querying the STAC API

In [ ]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "https://staging-stac.delta-backend.com"
RASTER_API_URL = "https://staging-raster.delta-backend.com"

# Declare collection of interest - NCEO Biomass
collection_name = "nceo_africa_2017"

Fetching the STAC collection and printing it, is one of the easiest and quickest ways to understand the metadata that makes up this collection

In [ ]:
import requests

#Fetch STAC collection
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

In [ ]:
# Get collection summary
collection["summaries"]

Printing the collection's `summaries` we see that the dataset is for 2017 only and that the `min` and `max` values for the observed values range from `0` to `429` Mg ha-1.

Returning back to our STAC API requests, let's check how many total items are available. 

In [ ]:
# Check total number of items available
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit=100").json()["features"]
print(f"Found {len(items)} items")


This makes sense is our collection is a single map for 2017, so we should only have 1 item.

In [ ]:
# Explore one item to see what it contains
items[0]

## Accessing the data

Now that we've explored the dataset through the STAC API, let's read and access the dataset itself. 

In [ ]:
from cartopy import crs as ccrs
import hvplot.xarray
import rasterio
import xarray as xr



s3_link = (items[0]['assets']['cog_default']['href'])
da = xr.open_dataset(s3_link, engine="rasterio")
da = da.squeeze("band", drop=True)
da

In [ ]:
# Create an AOI for our study area

# Guinea
guinea_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
          [
            [
              -15.519958756713947,
              12.732440363049193
            ],
            [
              -15.519958756713947,
              6.771426493209475
            ],
            [
              -7.078554695621165,
              6.771426493209475
            ],
            [
              -7.078554695621165,
              12.732440363049193
            ],
            [
              -15.519958756713947,
              12.732440363049193
            ]
          ]
        ],
        "type": "Polygon"
      }
    }

In [ ]:
# Subset to bounding box 
subset = da.rio.clip([guinea_aoi["geometry"]])
#subset